In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import glob

# Define the data directory and the figure directory
data_dir = "/Users/asca/Documents/University/Master Thesis/code/Data/VIGO17_NS089008_QPD_0750_20_AS_011_CC/20240613"
fig_dir = "/Users/asca/Documents/University/Master Thesis/code/Data/VIGO17_NS089008_QPD_0750_20_AS_011_CC/20240613/fig"

# Get all the .pkl files
file_list = sorted(glob.glob(os.path.join(data_dir, "*.pkl")))

# Function to load data from a .pkl file
def load_data(file_path):
    with open(file_path, "rb") as file:
        data = pickle.load(file)
    return data.get('rawdata', None)

# Define the custom colormap
colors = [(0, 0, 0), (1, 0, 0), (1, 0.5, 0), (1, 1, 1)]  # Black -> Red -> Orange -> White
n_bins = 100  # Number of bins for the colormap
cm = mcolors.LinearSegmentedColormap.from_list("black_red_white", colors, N=n_bins)

# Function to plot quadrants
def plot_quadrant(ax, x, y, data, title, cm):
    c = ax.pcolormesh(x, y, data, shading='auto', cmap=cm, vmin=0, vmax=np.max(data))
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel("X Position [µm]", fontsize=12)
    ax.set_ylabel("Y Position [µm]", fontsize=12)
    ax.set_aspect('equal')  # Ensure square aspect ratio
    
    # Set x-axis ticks to multiples of 500 microns within the data range
    x_min, x_max = np.min(x), np.max(x)
    x_ticks = np.arange(x_min, x_max + 1, 500)  # Multiples of 500 microns
    ax.set_xticks(x_ticks)
    
    # Set y-axis ticks to multiples of 250 microns within the data range
    y_min, y_max = np.min(y), np.max(y)
    y_ticks = np.arange(y_min, y_max + 1, 250)  # Multiples of 250 microns
    ax.set_yticks(y_ticks)
    
    # Set tick parameters for white ticks and black labels
    ax.tick_params(which='major', direction='in', top=True, right=True, color='white', labelcolor='black', length=8, width=1.5)  # Longer major ticks
    ax.tick_params(which='minor', direction='in', top=True, right=True, color='white', labelcolor='black', length=4, width=1)  # Shorter minor ticks
    ax.minorticks_on()
    
    # Remove grid lines
    ax.grid(False)
    
    # Add color bar
    cbar = plt.colorbar(c, ax=ax, shrink=0.8)
    cbar.set_label("DC Current [mA]", fontsize=12)
    cbar.set_ticks(np.arange(0, 0.26, 0.05))

# Loop through each .pkl file and generate plots
for file_path in file_list:
    rawdata = load_data(file_path)
    if rawdata is None:
        print(f"Skipping {file_path}, no 'rawdata' found.")
        continue
    
    # Extract position data
    x_positions = np.array(rawdata['stage_laser_xposition'])
    y_positions = np.array(rawdata['stage_laser_yposition'])
    
    # Determine grid shape dynamically
    grid_shape = (len(np.unique(x_positions)), len(np.unique(y_positions)))
    
    # Extract and process current data
    try:
        dccurr_A = np.array(rawdata['quadA']['dmm00_curr_amp']).mean(axis=2).flatten().reshape(grid_shape) * 1000
        dccurr_B = np.array(rawdata['quadB']['dmm00_curr_amp']).mean(axis=2).flatten().reshape(grid_shape) * 1000
        dccurr_C = np.array(rawdata['quadC']['dmm00_curr_amp']).mean(axis=2).flatten().reshape(grid_shape) * 1000
        dccurr_D = np.array(rawdata['quadD']['dmm00_curr_amp']).mean(axis=2).flatten().reshape(grid_shape) * 1000
    except ValueError:
        print(f"Skipping {file_path} due to unexpected data shape.")
        continue

    # Create figure with 4 subplots
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    plot_quadrant(axes[0, 0], x_positions, y_positions, dccurr_A, "Quadrant A", cm)
    plot_quadrant(axes[0, 1], x_positions, y_positions, dccurr_B, "Quadrant B", cm)
    plot_quadrant(axes[1, 0], x_positions, y_positions, dccurr_C, "Quadrant C", cm)
    plot_quadrant(axes[1, 1], x_positions, y_positions, dccurr_D, "Quadrant D", cm)
    
    plt.tight_layout()
    plt.show()
    
    # Save figure to the fig_dir
    filename = os.path.basename(file_path).replace(".pkl", ".png")
    save_path = os.path.join(fig_dir, filename)
    plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')  # Set background to white
    print(f"Saved: {save_path}")
    
    plt.close()  # Close figure to avoid excessive memory usage

print("All plots generated and saved successfully.")

Saved: /Users/asca/Documents/University/Master Thesis/code/Data/VIGO17_NS089008_QPD_0750_20_AS_011_CC/20240613/fig/20240613_150822_Run1_VIGO17_NS089008_QPD_20_20_AS_011_CC_AmplitudeUniformity.png
Saved: /Users/asca/Documents/University/Master Thesis/code/Data/VIGO17_NS089008_QPD_0750_20_AS_011_CC/20240613/fig/20240613_161943_Run1_VIGO17_NS089008_QPD_20_20_AS_011_CC_AmplitudeUniformity.png
Saved: /Users/asca/Documents/University/Master Thesis/code/Data/VIGO17_NS089008_QPD_0750_20_AS_011_CC/20240613/fig/20240613_165007_Run1_VIGO17_NS089008_QPD_20_20_AS_011_CC_AmplitudeUniformity.png
Saved: /Users/asca/Documents/University/Master Thesis/code/Data/VIGO17_NS089008_QPD_0750_20_AS_011_CC/20240613/fig/20240613_204151_Run1_VIGO17_NS089008_QPD_20_20_AS_011_CC_AmplitudeUniformity.png
All plots generated and saved successfully.
